# **Features Extraction (Text Vectorization)**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
from scipy.sparse import hstack
import os , pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Personal_Project/Feature_Engineering/Advance_Peprocessing _file/Preprocess_part_2.csv')
df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate,q1len,q2len,q1_n_words,q2_n_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,66,57,14,12,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,51,88,8,13,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,73,59,14,10,...,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,50,65,11,9,...,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,76,39,13,7,...,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [ ]:
#sampling 100k
df = df.sample(n=100000,random_state=40)

In [ ]:
#changing columns to numeric type
num_cols = df.drop(columns=['id', 'qid1', 'qid2', 'question1', 'question2']).columns
for i in num_cols:
    df[i] = df[i].apply(pd.to_numeric)

In [ ]:
y = df['is_duplicate']
X = df[df.drop(columns=['id', 'qid1', 'qid2','is_duplicate']).columns.tolist()]
print(X.shape)
print(y.shape)

(100000, 28)
(100000,)


In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3,random_state=100)
print("Number of data points in train data :",X_train.shape)
print("Number of data points in test data :",X_test.shape)

Number of data points in train data : (70000, 28)
Number of data points in test data : (30000, 28)


# **Word Embeding**

* TFIDF
* TFIDF weighted glove vectorization

In [ ]:
X_train['question1'] = X_train['question1'].dropna()

In [ ]:
tfidf_vectorizer1 = TfidfVectorizer(lowercase=False,max_features= 20000)
trainqs1_tfidf = tfidf_vectorizer1.fit_transform(X_train['question1'].values.astype('U'))
testqs1_tfidf  = tfidf_vectorizer1.transform(X_test['question1'].values.astype('U'))
print(trainqs1_tfidf.shape)
print(testqs1_tfidf.shape)

(70000, 20000)
(30000, 20000)


In [ ]:
tfidf_vectorizer2 = TfidfVectorizer(lowercase=False,max_features= 20000)
train_qs2_tfidf = tfidf_vectorizer2.fit_transform(X_train['question2'].values.astype('U'))
test_qs2_tfidf  = tfidf_vectorizer2.transform(X_test['question2'].values.astype('U'))
print(train_qs2_tfidf.shape)
print(test_qs2_tfidf.shape)

(70000, 20000)
(30000, 20000)


In [ ]:
#Now we will hstack both the vectors
tfidf_train_vec = hstack((trainqs1_tfidf,train_qs2_tfidf))
tfidf_test_vec = hstack((testqs1_tfidf,test_qs2_tfidf)) 
print("train data shape",tfidf_train_vec.shape)
print("Test data shape ",tfidf_test_vec.shape)

train data shape (70000, 40000)
Test data shape  (30000, 40000)


In [ ]:
# selecting other features
train_df = X_train.drop(columns=['question1', 'question2'])
test_df = X_test.drop(columns=['question1', 'question2'])

In [ ]:
#we need to convert our data with features into sparse matrix so that we can combine our feature matrix and and tfidf vectors 
import scipy
train_sparse = scipy.sparse.csr_matrix(train_df)
test_sparse = scipy.sparse.csr_matrix(test_df)

In [ ]:
# Now combining our tfidf and features into one 
tfidf_X_tr = hstack((train_sparse,tfidf_train_vec))
tfidf_X_test = hstack((test_sparse,tfidf_test_vec))
print("train data shape",tfidf_X_tr.shape)
print("Test data shape ",tfidf_X_test.shape)

train data shape (70000, 40026)
Test data shape  (30000, 40026)


In [ ]:
os.chdir('/content/drive/MyDrive/Personal_Project/Fetures_Extraction/TFIDF')

In [ ]:
#saving tfidf vectors
pickle.dump(tfidf_train_vec, open("tfidf_X_train","wb"))
pickle.dump(tfidf_test_vec, open("tfidf_X_test","wb"))